In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaneddataaaa/cleaned_trained_dataset.csv
/kaggle/input/testtrain/train.csv
/kaggle/input/testtrain/test.csv


In [2]:
# Install required packages
!pip install transformers>=4.51.0 torch torchvision torchaudio accelerate bitsandbytes -q
!pip install sentencepiece protobuf pandas -q

In [3]:
# Global variables
MODEL_NAME = "Qwen/Qwen3-14B"
LABELS = ["hope", "hate", "not_applicable"]
tokenizer = None
model = None

In [4]:
# Arabic Text Classifier using Qwen3-14B for Kaggle - Functional Version
# Make sure to enable GPU (T4 x2 recommended) in Kaggle notebook settings



import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
import numpy as np
import gc
import warnings
import re
from sklearn.metrics import classification_report, confusion_matrix
import json
warnings.filterwarnings('ignore')



def check_gpu():
    """Check GPU availability and print info"""
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

def load_model():
    """Load Qwen3-14B with optimizations for Kaggle and Arabic text"""
    global tokenizer, model
    
    print("Loading Qwen3-14B model for Arabic Text Classification...")
    
    # Configure quantization for memory efficiency on Kaggle
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        
    )
    
    # Load model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True
    )
    
    print("Qwen3-14B model loaded successfully!")
    print_model_info()

def print_model_info():
    """Print model and memory information"""
    print(f"\nModel Information:")
    print(f"Model Name: {MODEL_NAME}")
    print(f"Model Parameters: 14.8B (13.2B non-embedding)")
    print(f"Context Length: 32,768 tokens")
    print(f"Tokenizer Vocab Size: {len(tokenizer):,}")
    print(f"Classification Labels: {', '.join(LABELS)}")
    
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

def extract_classification(response):
    """Extract classification label from response text"""
    
    # Most specific patterns first - look for the actual Classification section
    patterns = [
        r'Classification:\s*\[\s*(hope|hate|not_applicable)\s*\]',  # Classification: [not_applicable]
        r'Classification:\s*\*\s*(hope|hate|not_applicable)\s*\*',  # Classification: *hope*
        r'Classification:\s*(hope|hate|not_applicable)\s*$',        # Classification: hope (end of line)
        r'Classification:\s*(hope|hate|not_applicable)',            # Classification: hope
        r'Category:\s*(hope|hate|not_applicable)',
        r'Label:\s*(hope|hate|not_applicable)',
        r'Result:\s*(hope|hate|not_applicable)',
        r'Final.*?:\s*(hope|hate|not_applicable)',
    ]
    
    response_lower = response.lower()
    
    # Try patterns in order of preference - take the LAST occurrence
    for pattern in patterns:
        matches = list(re.finditer(pattern, response_lower, re.IGNORECASE | re.MULTILINE))
        if matches:
            # Take the last match to avoid analysis section
            match = matches[-1]
            label = match.group(1).lower()
            if label in LABELS:
                confidence = estimate_confidence(response, label)
                return label, confidence
    
    # More conservative fallback - only look in the last part of response
    response_lines = response_lower.split('\n')
    last_lines = response_lines[-3:]  # Only check last 3 lines
    
    for line in reversed(last_lines):
        for label in LABELS:
            if f' {label}' in line or f'{label} ' in line or f'{label}.' in line:
                return label, 0.6
    
    # Final fallback - but exclude analysis/reasoning sections
    label_counts = {}
    # Split response and only count from classification section onwards
    classification_start = response_lower.find('classification')
    if classification_start != -1:
        relevant_text = response_lower[classification_start:]
    else:
        relevant_text = response_lower[-200:]  # Only last 200 chars
    
    for label in LABELS:
        count = len(re.findall(r'\b' + label + r'\b', relevant_text))
        if count > 0:
            label_counts[label] = count
    
    if label_counts:
        predicted_label = max(label_counts, key=label_counts.get)
        confidence = 0.4  # Lower confidence for fallback
        return predicted_label, confidence
    
    # Default fallback
    return "not_applicable", 0.3

def estimate_confidence(response, predicted_label):
    """Estimate confidence based on response characteristics"""
    confidence_indicators = [
        "clearly", "definitely", "obviously", "certain", "confident",
        "strong", "evident", "apparent", "明显", "确定", "肯定"
    ]
    
    uncertainty_indicators = [
        "might", "possibly", "perhaps", "maybe", "unclear",
        "difficult", "ambiguous", "uncertain", "可能", "也许", "不确定"
    ]
    
    response_lower = response.lower()
    
    confidence_score = 0.7  # Base confidence
    
    # Increase confidence if confident language is used
    for indicator in confidence_indicators:
        if indicator in response_lower:
            confidence_score += 0.1
            break
    
    # Decrease confidence if uncertain language is used
    for indicator in uncertainty_indicators:
        if indicator in response_lower:
            confidence_score -= 0.2
            break
    
    # Ensure confidence is in valid range
    confidence_score = max(0.1, min(0.95, confidence_score))
    
    return round(confidence_score, 2)

def classify_arabic_text(text, use_thinking_mode=False, max_new_tokens=8000):
    """
    Classify Arabic text into hope/hate/not_applicable categories
    
    Args:
        text (str): Arabic text to classify
        use_thinking_mode (bool): Enable Qwen3's thinking capabilities
        max_new_tokens (int): Maximum tokens to generate
    
    Returns:
        tuple: (full_response, predicted_label, confidence, thinking_content)
    """
    global tokenizer, model
    
    if tokenizer is None or model is None:
        raise ValueError("Model not loaded. Please call load_model() first.")
    
    # Format the conversation for Qwen3
    messages = [
            {
        "role": "user",
        "content": f"""Please classify the following Arabic text into one of these categories:

Categories:
1. "hope" - Text expressing *clear* hope, longing for positive change, admiration with intent, affection toward a person or idea, or constructive sentiment. Purely poetic descriptions or artistic metaphors should only be labeled "hope" if they convey **intentional optimism** or **emotional uplift**.
2. "hate" - Text expressing hate, negativity, hostility, discrimination, or destructive sentiment — including coded language, sarcasm, sectarian slogans, or historically divisive terms. However, sarcasm or satire **without a clear hostile target** should not be misclassified as hate.
3. "not_applicable" - Text that doesn't clearly fit into hope or hate categories, including neutral, factual, or purely aesthetic/poetic content **without clear hopeful or hateful intent**.

Instructions:
- Carefully analyze whether the **intent** behind the text expresses constructive or destructive sentiment.
- Do not classify admiration or personal preference for athletes, celebrities, or teams as "hope" unless there is a broader social or emotional message of uplift, change, or optimism.
- Do **not** label emotional or metaphorical language as "hope" unless there is a **clear purpose** or **intent** of expressing optimism, affection, or desire for positive change.
- **Spiritual or emotional self-criticism**, such as expressing a personal struggle with faith or oneself (e.g., “my heart fights God”), should be treated as **not_applicable** unless it directly targets a religious group or belief with hostility.
- **Sarcasm, irony, or meme language** (e.g., using emojis or exaggeration) should be **ignored** unless it clearly targets a group or person in a hostile or discriminatory way.

Comparison Between All labels:
Compare each pair by how they differ in emotional tone, intent, and expression:

neutral vs anger: Neutral is emotionless or factual; anger expresses irritation or hostility.

neutral vs anticipation: Neutral lacks expectation; anticipation expresses eagerness or hope.

neutral vs disgust: Neutral is objective; disgust reflects moral or physical aversion.

neutral vs fear: Neutral is calm; fear expresses anxiety or threat.

neutral vs joy: Neutral lacks emotion; joy is positive and energetic.

neutral vs love: Neutral is indifferent; love expresses affection and emotional closeness.

neutral vs optimism: Neutral is flat; optimism is hopeful about outcomes.

neutral vs pessimism: Neutral is unbiased; pessimism expects negative results.

neutral vs sadness: Neutral is unfeeling; sadness conveys sorrow or loss.

neutral vs surprise: Neutral is steady; surprise shows sudden reaction or disbelief.

neutral vs trust: Neutral is detached; trust reflects confidence or belief in reliability.

anger vs anticipation: Anger is hostile and reactive; anticipation is hopeful and forward-looking.

anger vs disgust: Anger reacts to provocation; disgust judges or rejects morally or physically.

anger vs fear: Anger confronts; fear avoids or retreats.

anger vs joy: Anger is negative and aggressive; joy is positive and uplifting.

anger vs love: Anger pushes away; love pulls closer with affection.

anger vs optimism: Anger is critical or frustrated; optimism expects positive outcomes.

anger vs pessimism: Both can be negative, but anger is active frustration; pessimism is passive hopelessness.

anger vs sadness: Anger is explosive; sadness is internalized and quiet.

anger vs surprise: Anger is predictable from offense; surprise is reaction to unexpected events.

anger vs trust: Anger breaks bonds; trust builds them.

anticipation vs disgust: Anticipation looks forward positively; disgust pushes away negatively.

anticipation vs fear: Anticipation is hopeful expectation; fear is anxious expectation.

anticipation vs joy: Anticipation is future-focused; joy is in the present.

anticipation vs love: Anticipation expects; love connects emotionally.

anticipation vs optimism: Anticipation expects a specific event; optimism is a general positive outlook.

anticipation vs pessimism: Anticipation is hopeful; pessimism is doubtful.

anticipation vs sadness: Anticipation excites; sadness dulls.

anticipation vs surprise: Anticipation expects; surprise is unexpected.

anticipation vs trust: Anticipation is forward-looking; trust is present confidence.
disgust vs fear: Disgust is moral or physical rejection; fear is about danger or harm.

disgust vs joy: Disgust is repelled; joy is attracted.

disgust vs love: Disgust rejects; love embraces.

disgust vs optimism: Disgust is negative evaluation; optimism is positive expectation.

disgust vs pessimism: Both are negative; disgust is reactive judgment; pessimism is negative belief.

disgust vs sadness: Disgust rejects others; sadness focuses on inner loss.

disgust vs surprise: Disgust is evaluative; surprise is reactive.

disgust vs trust: Disgust pushes away; trust builds closeness.
fear vs joy: Fear is anxiety; joy is happiness.

fear vs love: Fear is isolating; love is connecting.

fear vs optimism: Fear expects harm; optimism expects good.

fear vs pessimism: Both expect bad; fear is emotional and urgent, pessimism is mental and resigned.

fear vs sadness: Fear relates to danger; sadness relates to loss.

fear vs surprise: Fear is anticipating harm; surprise is sudden and undefined.

fear vs trust: Fear hesitates; trust commits.
joy vs love: Joy is delight in the moment; love is deep emotional attachment.

joy vs optimism: Joy is current happiness; optimism is future hope.

joy vs pessimism: Joy is positive; pessimism is negative.

joy vs sadness: Joy and sadness are opposites in emotional tone.

joy vs surprise: Joy is lasting; surprise is brief.

joy vs trust: Joy is feeling good; trust is feeling safe.
love vs optimism: Love is relational; optimism is outlook-based.

love vs pessimism: Love is warm and connecting; pessimism is cold and distant.

love vs sadness: Love is bonding; sadness often reflects loss of love.

love vs surprise: Love is expected over time; surprise is sudden.

love vs trust: Love is emotional closeness; trust is reliability and belief.

optimism vs pessimism: Optimism expects good; pessimism expects bad.

optimism vs sadness: Optimism looks forward positively; sadness reflects current or past loss.

optimism vs surprise: Optimism is long-term hope; surprise is momentary shock.

optimism vs trust: Optimism is about outcomes; trust is about people or systems.
pessimism vs sadness: Pessimism expects bad; sadness feels bad.

pessimism vs surprise: Pessimism expects negative; surprise reacts to unexpected (positive or negative).

pessimism vs trust: Pessimism doubts; trust believes.

sadness vs surprise: Sadness is deep and emotional; surprise is brief and reactive.

sadness vs trust: Sadness reflects pain; trust reflects faith.
surprise vs trust: Surprise is sudden reaction; trust is steady belief.
Offensive content includes insults, slurs, profanity, vulgar jokes, threats, sexually explicit or socially inappropriate language.
Non-offensive content is respectful, civil, and appropriate—even if emotionally intense (e.g., sadness, fear, or anger), it avoids harmful or vulgar expression.

Hate content targets identity groups (e.g., race, religion, nationality, gender, sexuality) with derogatory, dehumanizing, or violent language.
Not_hate content may still be offensive (e.g., personal insults, rude jokes, spam, crude humor), but does not target individuals or groups based on identity.


Arabic Text: {text}

Provide your response in this format:
- Analysis: [Explain what the text is saying and whether it shows hostility, admiration, or neutrality]
- Reasoning: [Explain your classification decision]
- Classification: [hope/hate/not_applicable]
"""

    }
    ]

    
    # Apply chat template with thinking mode control
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=use_thinking_mode
    )
    
    # Tokenize input
    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)
    
    # Generate response with optimized parameters for Qwen3
    with torch.no_grad():
        if use_thinking_mode:
            # Thinking mode parameters
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.6,
                top_p=0.7,
                top_k=20,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        else:
            # Non-thinking mode parameters
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                top_p=0.8,
                top_k=20,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
    
    # Extract output tokens
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    
    # Parse thinking content (if thinking mode is enabled)
    thinking_content = ""
    content = ""
    
    if use_thinking_mode:
        try:
            # Find </think> token (151668)
            index = len(output_ids) - output_ids[::-1].index(151668)
            thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
            content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
        except ValueError:
            # No thinking tags found
            content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    else:
        content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    # Extract the classification and confidence
    predicted_label, confidence = extract_classification(content)
    
    return content, predicted_label, confidence, thinking_content

def cleanup_memory():
    """Clean up GP memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def process_dataset(df, text_column='text', id_column='id', 
                   use_thinking=True, batch_size=10, save_progress=True):
    """
    Process entire dataset for classification
    
    Args:
        df (pd.DataFrame): Dataset with Arabic text
        text_column (str): Name of text column
        id_column (str): Name of ID column  
        use_thinking (bool): Enable thinking mode
        batch_size (int): Process in batches to manage memory
        save_progress (bool): Save intermediate results
    
    Returns:
        pd.DataFrame: Results with predictions
    """
    
    results = []
    total_samples = len(df)
    
    print(f"🚀 Processing {total_samples} Arabic text samples...")
    print(f"📊 Batch size: {batch_size}")
    print("-" * 60)
    
    for i in range(0, total_samples, batch_size):
        batch_end = min(i + batch_size, total_samples)
        batch_df = df.iloc[i:batch_end]
        
        print(f"Processing batch {i//batch_size + 1}/{(total_samples-1)//batch_size + 1} (samples {i+1}-{batch_end})")
        
        batch_results = []
        
        for idx, row in batch_df.iterrows():
            try:
                text = row[text_column]
                sample_id = row[id_column]
                
                # Classify the text
                response, predicted_label, confidence, thinking = classify_arabic_text(
                    text, use_thinking_mode=use_thinking, max_new_tokens=1500
                )
                
                result = {
                    'id': sample_id,
                    'text': text,
                    'predicted_label': predicted_label,
                    'confidence': confidence,
                    'full_response': response,
                    'thinking_process': thinking if use_thinking else ""
                }
                
                # Add actual label if exists
                if 'label' in row:
                    result['actual_label'] = row['label']
                
                batch_results.append(result)
                print(f"  ✅ Sample {sample_id}: {predicted_label} (confidence: {confidence})")
                
            except Exception as e:
                print(f"  ❌ Error processing sample {row[id_column]}: {str(e)}")
                batch_results.append({
                    'id': row[id_column],
                    'text': row[text_column],
                    'predicted_label': 'not_applicable',
                    'confidence': 0.1,
                    'full_response': f"Error: {str(e)}",
                    'thinking_process': ""
                })
        
        results.extend(batch_results)
        
        # Clean up memory after each batch
        cleanup_memory()
        
        # Save intermediate progress
        if save_progress and (i + batch_size) % (batch_size * 5) == 0:
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f'intermediate_results_batch_{i//batch_size + 1}.csv', index=False)
            print(f"  💾 Saved intermediate results up to batch {i//batch_size + 1}")
        
        print(f"  ✅ Batch completed. Memory cleaned.")
        print("-" * 40)
    
    results_df = pd.DataFrame(results)
    
    print(f"🎉 Classification completed!")
    print(f"📊 Total processed: {len(results_df)} samples")
    
    return results_df

def evaluate_predictions(results_df):
    """Evaluate predictions if actual labels are available"""
    
    if 'actual_label' not in results_df.columns:
        print("⚠️ No actual labels found for evaluation")
        return None
    
    # Filter out rows with missing actual labels
    eval_df = results_df.dropna(subset=['actual_label'])
    
    if len(eval_df) == 0:
        print("⚠️ No valid actual labels found for evaluation")
        return None
    
    y_true = eval_df['actual_label'].tolist()
    y_pred = eval_df['predicted_label'].tolist()
    
    print("📊 Classification Report:")
    print("=" * 60)
    print(classification_report(y_true, y_pred, target_names=LABELS))
    
    print("\n📊 Confusion Matrix:")
    print("=" * 30)
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    cm_df = pd.DataFrame(cm, index=LABELS, columns=LABELS)
    print(cm_df)
    
    # Calculate accuracy by label
    print("\n📊 Accuracy by Label:")
    print("=" * 30)
    for label in LABELS:
        label_mask = eval_df['actual_label'] == label
        if label_mask.sum() > 0:
            accuracy = (eval_df[label_mask]['predicted_label'] == label).mean()
            print(f"{label}: {accuracy:.3f} ({label_mask.sum()} samples)")
    
    # Overall accuracy
    overall_accuracy = (eval_df['actual_label'] == eval_df['predicted_label']).mean()
    print(f"\nOverall Accuracy: {overall_accuracy:.3f}")
    
    return {
        'accuracy': overall_accuracy,
        'classification_report': classification_report(y_true, y_pred, target_names=LABELS, output_dict=True),
        'confusion_matrix': cm_df
    }

def load_and_process_dataset(file_path, text_column='text', id_column='id', 
                           label_column='label', use_thinking=True):
    """
    Load and process the Arabic dataset
    
    Args:
        file_path (str): Path to CSV file
        text_column (str): Name of text column
        id_column (str): Name of ID column
        label_column (str): Name of label column (optional)
        use_thinking (bool): Enable thinking mode
    
    Returns:
        tuple: (original_df, results_df, evaluation_results)
    """
    
    # Load dataset
    print(f"📂 Loading dataset from: {file_path}")
    df = pd.read_csv(file_path)
    
    print(f"📊 Dataset shape: {df.shape}")
    print(f"📋 Columns: {list(df.columns)}")
    
    # Show sample data
    print("\n📝 Sample data:")
    print(df.head())
    
    # Check label distribution if available
    if label_column in df.columns:
        print(f"\n📊 Label distribution:")
        print(df[label_column].value_counts())
    
    # Process the dataset
    results_df = process_dataset(
        df, 
        text_column=text_column, 
        id_column=id_column, 
        use_thinking=use_thinking,
        batch_size=5,  # Small batch size for memory management
        save_progress=True
    )
    
    # Evaluate if possible
    evaluation_results = None
    if label_column in df.columns:
        results_df['actual_label'] = df[label_column]
        evaluation_results = evaluate_predictions(results_df)
    
    return df, results_df, evaluation_results

def classify_single_text(text, show_details=True):
    """Classify a single Arabic text"""
    
    print("🔍 Single Text Classification")
    print("=" * 50)
    print(f"Arabic Text: {text}")
    print("-" * 50)
    
    response, predicted_label, confidence, thinking = classify_arabic_text(
        text, use_thinking_mode=True
    )
    
    if show_details and thinking:
        print("🧠 Thinking Process:")
        print(thinking)
        print("\n" + "-" * 40)
    
    print("📝 Analysis:")
    print(response)
    
    print(f"\n✅ Classification: {predicted_label}")
    print(f"🎯 Confidence: {confidence}")
    print("=" * 50)
    
    return predicted_label, confidence

def run_example_classifications():
    """Run example Arabic text classifications"""
    
    example_texts = [
        "أتمنى أن يكون الغد أفضل من اليوم",  # Hope: I hope tomorrow will be better than today
        "أكره هذا الشيء كثيراً",  # Hate: I hate this thing so much  
        "الطقس اليوم مشمس",  # Not applicable: The weather is sunny today
        "سأحارب من أجل المستقبل الأفضل",  # Hope: I will fight for a better future
        "هؤلاء الناس لا يستحقون الاحترام",  # Hate: These people don't deserve respect
        "كم الساعة الآن؟"  # Not applicable: What time is it now?
    ]
    
    print("🚀 Running Example Arabic Text Classifications")
    print("=" * 60)
    
    for i, text in enumerate(example_texts, 1):
        print(f"\nExample {i}:")
        classify_single_text(text, show_details=False)

def save_results(results_df, filename="arabic_classification_results.csv"):
    """Save classification results"""
    results_df.to_csv(filename, index=False, encoding='utf-8')
    print(f"💾 Results saved to: {filename}")
    
    # Also save a summary
    summary = {
        'total_samples': len(results_df),
        'label_distribution': results_df['predicted_label'].value_counts().to_dict(),
        'avg_confidence': results_df['confidence'].mean(),
        'confidence_by_label': results_df.groupby('predicted_label')['confidence'].mean().to_dict()
    }
    
    with open(filename.replace('.csv', '_summary.json'), 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)
    
    print(f"📊 Summary saved to: {filename.replace('.csv', '_summary.json')}")

# Initialize the system 
print("🚀 Arabic Text Classifier (Functional Version) ready!")

# Check GPU and load model
check_gpu()
load_model()

print("📚 Running example classifications...")

# Example usage:
# run_example_classifications()

# To use with your dataset:
# df, results_df, eval_results = load_and_process_dataset('your_dataset.csv')
# save_results(results_df

🚀 Arabic Text Classifier (Functional Version) ready!
CUDA available: True
GPU: Tesla P100-PCIE-16GB
GPU Memory: 15.9 GB
Loading Qwen3-14B model for Arabic Text Classification...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3-14B model loaded successfully!

Model Information:
Model Name: Qwen/Qwen3-14B
Model Parameters: 14.8B (13.2B non-embedding)
Context Length: 32,768 tokens
Tokenizer Vocab Size: 151,669
Classification Labels: hope, hate, not_applicable
GPU Memory Allocated: 9.27 GB
GPU Memory Reserved: 11.00 GB
📚 Running example classifications...


In [5]:
text='مختصر دين الوهابية حاجات معاوية_رأس_الفتنة'
classify_single_text(text, show_details=False)

🔍 Single Text Classification
Arabic Text: مختصر دين الوهابية حاجات معاوية_رأس_الفتنة
--------------------------------------------------
📝 Analysis:
- Analysis: The text references "Wahabi religion" (الوهابية) and links it to Muawiya ibn Abi Sufyan (معاوية), a historical figure associated with the First Fitna (a period of civil strife in early Islamic history). The phrase "رأس الفتنة" ("head of the Fitna") carries a strongly negative connotation, implying blame for division or conflict. The text appears to criticize or delegitimize Wahhabism through its association with a controversial historical figure.  
- Reasoning: The text explicitly connects a specific religious ideology (Wahhabism) to a historical figure (Muawiya) framed as a source of division ("رأس الفتنة"). This association implies hostility or disdain toward Wahhabism, aligning with the "hate" category due to its discriminatory or critical tone toward a religious group. While the phrasing is concise, the intent to associate a

('hate', 0.8)

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/testrain/test.csv')

In [ ]:
cnt=0

for i in range(40):
    text = df['text'][i]
    prediction,confidence = classify_single_text(text, show_details=False)
    print(prediction)
    print(df['label'][i])
    if prediction == df['label'][i]:
        cnt+=1

In [ ]:
print(cnt)